# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
# run end to end
# fix hdbscan with existing db somehow


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

11:13:49 | NewsletterAgent.newsletter_agent | INFO | Test info message
11:13:49 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
11:13:49 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [8]:
print("🚀 Creating NewsletterAgent...")

do_download=False
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251009073511683688'
    step_name = 'step_07_select_sections'
#     del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

11:15:48 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Using provided state with 146 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20251009073511683688
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20251009073511683688


In [9]:
agent.state.get_status()


{'headlines': {'total': 146},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_08_draft_sections',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': 'Categories and article counts:\ncat\nAI Adoption And Funding Surge    34\nAI Business Value Gap            12\nAI Lawsuit Settlement Funding     2\nAI Legal Services                 4\nAI Music Industry Disruption      2\nAI Societal Impact               42\nAgent Memory Frameworks           3\nChatGPT Apps Platform Play        2\nChatGPT Asia Expansion            2\nEU AI Sovereignty Strategy        3\nInstant AI Video Generators       4\nOpenAI AgentKit Launch            4\nOpenAI Massive Compute Deals      4\nOther                            20\nTSMC AI Revenue Boom              3\nThreat Actors Use AI Tools        2\nUS AI Policy Patchwork            3',
  'progress_percentage': 77.7777777777777

In [16]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


07:35:17 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Starting check_workflow_status
07:35:17 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Completed check_workflow_status


⏱️  Total execution time: 12.97s
📊 Final result:
Current workflow status:

- Progress: 0.0% (0/9 complete)
- Status summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Gather URLs (Fetch Urls)

Detailed steps:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Rate Articles — not_started
- Step 6: Cluster By Topic — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

What would you like me to do next?
- “Run all steps” or “Create the newsletter” — I will execute Steps 1–9 in sequence.
- “Run Step 1” — I will fetch URLs now.
- “Resume” or “Continue” — same as running the next incomplete step (Step 1).
- “Inspect state” — I can show the detailed persistent state for debugging.

Tell me which option you want.


In [17]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls, if any failed state which one and the url"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls, if any failed state which one and the url'


07:35:29 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Starting check_workflow_status
07:35:29 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Completed check_workflow_status
07:35:30 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Starting Step 1: Gather URLs
2025-10-09 07:35:30,856 - fetcher_5176075792 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-10-09 07:35:30,873 - fetcher_5176075792 - INFO - [fetcher_init] Loaded 17 sources: 6 RSS, 10 HTML, 1 API
2025-10-09 07:35:30,874 - fetcher_5176075792 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-10-09 07:35:30,874 - fetcher_5176075792 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-10-09 07:35:30,875 - fetcher_5176075792 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-10-09 07:35:30,876 - fetcher_5176075792 - DEB

▶ Starting Step 1: step_01_fetch_urls


2025-10-09 07:35:31,079 - fetcher_5176075792 - INFO - [fetch_html] Parsed HTML file: download/sources/Reddit.html
2025-10-09 07:35:31,079 - fetcher_5176075792 - INFO - [fetch_html] HTML fetch successful for Reddit: 63 articles
2025-10-09 07:35:31,080 - fetcher_5176075792 - INFO - [fetch_html] Using existing HTML file from Techmeme: https://www.techmeme.com/river
2025-10-09 07:35:31,080 - fetcher_5176075792 - INFO - [fetch_html] Parsing HTML file: download/sources/Techmeme.html
2025-10-09 07:35:31,102 - fetcher_5176075792 - INFO - [fetch_html] Parsed HTML file: download/sources/Techmeme.html
2025-10-09 07:35:31,103 - fetcher_5176075792 - INFO - [fetch_html] HTML fetch successful for Techmeme: 167 articles
2025-10-09 07:35:31,103 - fetcher_5176075792 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-10-09 07:35:31,104 - fetcher_5176075792 - INFO - [fetch_html] Using existing HTML file from The Verge: https://www.theverge.c

,source,url
0,Ars Technica,20
1,Bloomberg,27
2,Business Insider,16
3,FT,47
4,Feedly AI,71
5,Hacker News,29
6,HackerNoon,50
7,New York Times,23
8,NewsAPI,94
9,Reddit,52


,source,title,url,published,rss_summary,id
0,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,0
1,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,1
2,Ars Technica,Insurers balk at paying out huge settlements f...,https://arstechnica.com/ai/2025/10/insurers-ba...,"Wed, 08 Oct 2025 13:31:28 +0000","OpenAI, Anthropic consider using investor fund...",2
3,Ars Technica,Ars Live: Is the AI bubble about to pop? Ed Zi...,https://arstechnica.com/ai/2025/10/ars-live-is...,"Tue, 07 Oct 2025 19:15:26 +0000","Our discussion happens today, October 7, at 3:...",3
4,Ars Technica,Dead celebrities are apparently fair game for ...,https://arstechnica.com/ai/2025/10/dj-bruce-le...,"Tue, 07 Oct 2025 17:08:05 +0000","OpenAI's likeness protections for living ""publ...",4
...,...,...,...,...,...,...
714,NewsAPI,Fundamentals Beat Frenzy for FinTech IPOs in 2025,http://www.pymnts.com/news/ipo/2025/fundamenta...,2025-10-08T08:00:24Z,NaN,714
715,NewsAPI,Teck Cuts Copper Output at Mine That’s Key To ...,https://financialpost.com/pmn/business-pmn/tec...,2025-10-08T07:45:41Z,NaN,715
716,NewsAPI,AI Power Boom Turns Nuclear Stocks Into the Ma...,https://financialpost.com/pmn/business-pmn/ai-...,2025-10-08T09:22:42Z,NaN,716
717,NewsAPI,US lawmakers call for broader bans on chipmaki...,https://nypost.com/2025/10/08/business/us-lawm...,2025-10-08T08:54:01Z,NaN,717


07:35:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Completed Step 1: Gathered 825 articles


⏱️  Total execution time: 30.97s
📊 Final result:
I followed the workflow rules and checked status, then ran Step 1 (Fetch Urls).

Results of Step 1:
- Step 1 completed successfully.
- Sources scanned: 17 (RSS only).
- Articles gathered: 825
- Articles stored in persistent state: 719
- Difference (gathered − stored): 106 items not stored

About failed items:
- The workflow state shows the totals above but does not contain a per-URL failure log I can access from the current inspection output. In other words, I can confirm 106 gathered items were not stored, but the persistent state does not list which specific URLs failed or their error details.

Current workflow position:
- Current Step (next): Step 2 — Filter URLs to AI-related content

Options — what would you like me to do next?
- I can run Step 2 (filter URLs) now.
- I can produce a detailed listing of the 719 stored headlines and URLs (so you can review what was saved).
- I can attempt to re-run Step 1 to try to re-gather missing i

In [18]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,0
1,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,1
2,Ars Technica,Insurers balk at paying out huge settlements f...,https://arstechnica.com/ai/2025/10/insurers-ba...,"Wed, 08 Oct 2025 13:31:28 +0000","OpenAI, Anthropic consider using investor fund...",2
3,Ars Technica,Ars Live: Is the AI bubble about to pop? Ed Zi...,https://arstechnica.com/ai/2025/10/ars-live-is...,"Tue, 07 Oct 2025 19:15:26 +0000","Our discussion happens today, October 7, at 3:...",3
4,Ars Technica,Dead celebrities are apparently fair game for ...,https://arstechnica.com/ai/2025/10/dj-bruce-le...,"Tue, 07 Oct 2025 17:08:05 +0000","OpenAI's likeness protections for living ""publ...",4
...,...,...,...,...,...,...
714,NewsAPI,Fundamentals Beat Frenzy for FinTech IPOs in 2025,http://www.pymnts.com/news/ipo/2025/fundamenta...,2025-10-08T08:00:24Z,NaN,714
715,NewsAPI,Teck Cuts Copper Output at Mine That’s Key To ...,https://financialpost.com/pmn/business-pmn/tec...,2025-10-08T07:45:41Z,NaN,715
716,NewsAPI,AI Power Boom Turns Nuclear Stocks Into the Ma...,https://financialpost.com/pmn/business-pmn/ai-...,2025-10-08T09:22:42Z,NaN,716
717,NewsAPI,US lawmakers call for broader bans on chipmaki...,https://nypost.com/2025/10/08/business/us-lawm...,2025-10-08T08:54:01Z,NaN,717


In [19]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,153
8,NewsAPI,94
4,Feedly AI,71
9,Reddit,52
12,The Register,50
6,HackerNoon,50
3,FT,47
5,Hacker News,29
1,Bloomberg,27
7,New York Times,23


In [20]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


07:36:09 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Starting Step 2: Filter URLs
07:36:09 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | 🔍 Filtering 719 headlines...
07:36:09 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
07:36:09 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | 🔍 Filtering 719 articles for dupes.
07:36:09 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | URL deduplication: 356 duplicates filtered, 363 new URLs remain
07:36:09 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | 🔍 Filtering 719 headlines for AI relevance using LLM...


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/vandals-deface-ads-for-ai-necklaces-that-listen-to-all-your-conversations/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/insurers-balk-at-paying-out-huge-settlements-for-claims-against-ai-firms/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/ars-live-is-the-ai-bubble-about-to-pop-a-live-chat-with-ed-zitron/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/dj-bruce-lee-and-jackass-mr-rogers-dead-celebrities-become-puppets-in-sora-2-videos/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/openai-wants-to-make-chatgpt-into-a-universal-app-frontend/
found url, no cutof

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954
07:36:10 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Processing 15 chunks with concurrency 16


concurrency:  16


07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Completed Step 2: 181 AI-related articles, 356 duplicates removed


⏱️  Total execution time: 21.30s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 356 duplicate URLs, classified 363 new articles, found 181 AI-related.


In [21]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Starting Step 3: Download Articles
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Starting concurrent scraping of 181 AI-related articles


▶ Starting Step 3: step_03_download_articles


07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Launching browser for 181 URLs with 16 concurrent workers
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Worker 0 fetching 1 of 181 https://venturebeat.com/ai/new-memory-framework-builds-ai-agents-that-can-handle-the-real-worlds
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | scrape_url(https://venturebeat.com/ai/new-memory-framework-builds-ai-agents-that-can-handle-the-real-worlds)
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | scraping https://venturebeat.com/ai/new-memory-framework-builds-ai-agents-that-can-handle-the-real-worlds to download/html
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Downloading https://venturebeat.com/ai/new-memory-framework-builds-ai-agents-that-can-handle-the-real-worlds
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Worker 1 fetching 2 of 181 https://econ

07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Downloading https://www.npr.org/2025/10/08/nx-s1-5561981/ai-students-schools-teachers
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Worker 11 fetching 12 of 181 https://www.macworld.com/article/2935321/siri-anniversary-apple-intelligence-future-devices.html
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | scrape_url(https://www.macworld.com/article/2935321/siri-anniversary-apple-intelligence-future-devices.html)
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | scraping https://www.macworld.com/article/2935321/siri-anniversary-apple-intelligence-future-devices.html to download/html
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Downloading https://www.macworld.com/article/2935321/siri-anniversary-apple-intelligence-future-devices.html
07:36:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Worker 12

07:37:03 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Worker 11 fetching 18 of 181 https://t.co/ZF1E09CAyV
07:37:03 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | scrape_url(https://t.co/ZF1E09CAyV)
07:37:03 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | scraping https://t.co/ZF1E09CAyV to download/html
07:37:03 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Downloading https://t.co/ZF1E09CAyV
07:37:04 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Response: 200
07:37:05 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Saving HTML to download/html/AI_promises_efficiency_and_inclusion_but_needs_strong_oversight__RBI_Dy_Governor_Sankar.html
07:37:05 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Worker 1 completed https://economictimes.indiatimes.com/tech/artificial-intelligence/ai-promises-efficiency-and-inclusion-but-needs-strong-oversight-rbi-dy-governor-sankar/articl

...

concurrency:  16


07:46:25 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Inserted 7 new site records
07:46:25 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Updated 178 URL records with final URLs


Starting with 178 rows...
Processing 178 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

07:46:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Completed Step 3: Downloaded 165 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 36 vs 70
    36: Feedly AI - I Built an AI Prompt to Fight Post-Holiday Anxiety—And It Actually Works
    70: HackerNoon - I Built an AI Prompt to Fight Post-Holiday Anxiety—And It Actually Works
  Pair: 73 vs 38
    73: HackerNoon - The Machines That Make Music: Inside the Global Boom of AI-Generated Songs
    38: Feedly AI - The Machines That Make Music: Inside the Global Boom of AI-Generated Songs
  Pair: 35 vs 69
    35: Feedly AI - The Silent Revolution: AI-Driven Network Decisions in Real-Time
    69: HackerNoon - The Silent Revolution: AI-Driven Network Decisions in Real-Time
  Pair: 14 vs 143
    14: Feedly AI - Prediction: ThisArtificial Intelligence(AI) Stock Will Be the Nvidia of Quantum Computing by 2035
    143: NewsAPI - Prediction: This Artificial Intelligence (AI) Stock Will Be the Nvidia of Quantum Computing by 2035
  Pair: 34 vs 68
    34: Feedly AI - Carrying

In [22]:
headline_df = state.headline_df
headline_df.loc[headline_df["status"]!=200]



,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation
8,HackerNoon,I Built an AI Prompt to Fight Post-Holiday Anx...,https://hackernoon.com/i-built-an-ai-prompt-to...,"Thu, 09 Oct 2025 06:39:16 GMT","Post-holiday syndrome is that mix of anxiety, ...",216,True,exists,https://hackernoon.com/i-built-an-ai-prompt-to...,download/html/I_Built_an_AI_Prompt_to_Fight_Po...,None,download/text/I_Built_an_AI_Prompt_to_Fight_Po...,15456,hackernoon.com,Hacker Noon,2.0
18,HackerNoon,The Silent Revolution: AI-Driven Network Decis...,https://hackernoon.com/the-silent-revolution-a...,"Thu, 09 Oct 2025 07:26:27 GMT",The telecommunications industry is experiencin...,215,True,exists,https://hackernoon.com/the-silent-revolution-a...,download/html/The_Silent_Revolution__AI-Driven...,None,download/text/The_Silent_Revolution__AI-Driven...,10252,hackernoon.com,Hacker Noon,2.0
52,HackerNoon,Carrying Your Personal Memory Across AI Models...,https://hackernoon.com/carrying-your-personal-...,"Thu, 09 Oct 2025 07:27:17 GMT",Wwitching between models isn’t the hard part —...,214,True,exists,https://hackernoon.com/carrying-your-personal-...,download/html/Carrying_Your_Personal_Memory_Ac...,None,download/text/Carrying_Your_Personal_Memory_Ac...,5603,hackernoon.com,Hacker Noon,2.0
123,HackerNoon,"OpenAI’s Agent Builder Won’t Kill Zapier, Make...",https://hackernoon.com/openais-agent-builder-w...,"Thu, 09 Oct 2025 05:59:47 GMT",Does OpenAI's Agent Builder pose a threat to n...,218,True,exists,https://hackernoon.com/openais-agent-builder-w...,download/html/OpenAI_s_Agent_Builder_Won_t_Kil...,None,download/text/OpenAI_s_Agent_Builder_Won_t_Kil...,1765,hackernoon.com,Hacker Noon,2.0
153,Feedly AI,"AI to Consume 12 Percent of Electricity, but T...",https://www.inc.com/chloe-aiello/ai-to-consume...,NaN,NaN,165,True,403,https://www.inc.com/chloe-aiello/ai-to-consume...,,None,,0,inc.com,Inc.,1.0
154,Feedly AI,AI has carried the stock market. Concerns are ...,https://www.cnn.com/2025/10/09/investing/us-st...,NaN,NaN,170,True,skipped,https://www.cnn.com/2025/10/09/investing/us-st...,,None,,0,cnn.com,CNN,1.0
155,Bloomberg,AI Agent Startup N8n Nets $2.5 Billion Valuati...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,24,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
156,Bloomberg,US Approves Some Nvidia UAE Sales in Trump AI ...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,25,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
157,Techmeme,"TSMC reports Q3 revenue up 30% YoY to ~$32.5B,...",https://www.reuters.com/world/asia-pacific/tsm...,NaN,NaN,349,True,401,https://www.reuters.com/world/asia-pacific/tsm...,,None,,0,reuters.com,Reuters,2.0
158,Bloomberg,China Stocks Rise Post-Break as AI Buzz Outshi...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,26,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0


In [23]:
headline_df = state.headline_df
headline_df.loc[headline_df["html_path"]==""]



,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation
153,Feedly AI,"AI to Consume 12 Percent of Electricity, but T...",https://www.inc.com/chloe-aiello/ai-to-consume...,NaN,NaN,165,True,403,https://www.inc.com/chloe-aiello/ai-to-consume...,,None,,0,inc.com,Inc.,1.0
154,Feedly AI,AI has carried the stock market. Concerns are ...,https://www.cnn.com/2025/10/09/investing/us-st...,NaN,NaN,170,True,skipped,https://www.cnn.com/2025/10/09/investing/us-st...,,None,,0,cnn.com,CNN,1.0
155,Bloomberg,AI Agent Startup N8n Nets $2.5 Billion Valuati...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,24,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
156,Bloomberg,US Approves Some Nvidia UAE Sales in Trump AI ...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,25,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
157,Techmeme,"TSMC reports Q3 revenue up 30% YoY to ~$32.5B,...",https://www.reuters.com/world/asia-pacific/tsm...,NaN,NaN,349,True,401,https://www.reuters.com/world/asia-pacific/tsm...,,None,,0,reuters.com,Reuters,2.0
158,Bloomberg,China Stocks Rise Post-Break as AI Buzz Outshi...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,26,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
159,Techmeme,Sources: Microsoft is planning a major healthc...,https://www.wsj.com/tech/ai/microsoft-healthca...,NaN,NaN,351,True,skipped,https://www.wsj.com/tech/ai/microsoft-healthca...,,None,,0,wsj.com,The Wall Street Journal,5.0
160,Techmeme,Sources: Tesla abandoned Elon Musk's goal to m...,https://www.theinformation.com/articles/elon-m...,NaN,NaN,412,True,403,https://www.theinformation.com/articles/elon-m...,,None,,0,theinformation.com,The Information,2.0
161,Bloomberg,‘I Believe It’s a Bubble’: What Some Smart Peo...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,23,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
162,Techmeme,"Michel Devoret, a Google Quantum AI chief scie...",https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,405,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0


In [24]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

07:46:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Processing 169 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
07:46:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Using model 'gpt-4.1-mini' for summarization
07:46:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Starting summarization for 169 articles


▶ Starting Step 4: step_04_extract_summaries
concurrency:  16


07:46:31 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Processing 169 chunks with concurrency 16
07:46:58 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Extracting metadata from HTML files for 169 articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
INFO:llm:Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, system_len=1365, user_len=151
07:47:02 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Processing 169 chunks with concurrency 16


concurrency:  16
⏱️  Total execution time: 162.00s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 169/169 articles.
💾 Summaries stored in headline DataFrame.


In [ ]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



In [ ]:
# remove
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [25]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


07:49:13 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Rating 169 AI articles using fn_rate_articles
07:49:13 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Calculating article ratings for 169 articles
07:49:13 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Rating recency
07:49:13 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Rating spam probability
07:49:13 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Initialized LangfuseClient
07:49:13 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
07:49:13 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles
concurrency:  16


07:49:20 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | low quality articles: {0.0: 137, 1.0: 12, 0.8807969943837407: 2, 1.493094676197164e-10: 1, 0.6224592556713684: 1, 2.1024338523818466e-07: 1, 3.535625324942473e-05: 1, 0.9149009412616036: 1, 0.9875682652714307: 1, 0.008577484930474293: 1, 0.0003353500342255387: 1, 0.0001233946226544279: 1, 0.9997964803243284: 1, 0.6513548793721767: 1, 0.9706877045032167: 1, 0.835483478475751: 1, 0.9796676595602861: 1, 2.4616969518153846e-10: 1, 1.3615261080896538e-13: 1, 0.8175744308728488: 1, 2.8823507934443873e-13: 1}
07:49:20 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
07:49:20 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


concurrency:  16


07:49:30 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | on topic articles: {1.0: 123, 0.998073226003552: 2, 4.944450501938644e-09: 2, 0.9999416367033446: 2, 0.9999993295729128: 2, 0.9999996871837232: 1, 2.8823507934443873e-13: 1, 0.9999920581810099: 1, 0.9740425389805024: 1, 0.9999645227325883: 1, 0.4073333999755006: 1, 0.006692851264215723: 1, 0.9999962302845802: 1, 0.9999909853565219: 1, 1.4738974314670568e-05: 1, 0.010986941583869245: 1, 0.9999869324773292: 1, 0.008577484930474293: 1, 0.2689414052760013: 1, 0.798186813186023: 1, 0.835483478475751: 1, 0.867035682761771: 1, 0.18242550282051248: 1, 0.03732687913383872: 1, 2.289734845645553e-11: 1, 0.0: 1, 0.9999983759447105: 1, 0.0001233946226544279: 1, 2.2603250957489616e-06: 1, 0.6791786762514135: 1, 0.9999989719621736: 1, 0.8175744308728488: 1, 6.605216142043291e-05: 1, 0.9999756083407958: 1, 0.49999999904767284: 1, 0.9995694234965996: 1, 0.9999995679801056: 1, 0.07585817148650026: 1, 0.002472623024627213: 1, 0.9998

...

,id,title,bradley_terry,bt_z
2,2,An overview of detailed AI usage reports from ...,11.228139,2.297872
55,55,AMD’s Landmark Deal with OpenAI Marks a New Ph...,10.536909,2.156410
80,80,Sam Altman says that ChatGPT has reached 800M ...,8.973914,1.836538
74,74,"As part of its deal with AMD, OpenAI will rece...",8.724972,1.785592
118,118,EU wants key sectors to use made-in-Europe AI,8.295189,1.697635
...,...,...,...,...
77,77,How an H-1B visa-holder went from dental schoo...,-8.271533,-1.692794
60,60,China Stocks Rise Post-Break as AI Buzz Outshi...,-8.473999,-1.734229
140,140,Ocean Protocol withdraws from AI token allianc...,-8.937468,-1.829079
41,41,‘I Believe It’s a Bubble’: What Some Smart Peo...,-9.339768,-1.911411


08:09:02 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Top 10 ids: [2, 55, 80, 74, 118, 31, 98, 64, 132, 120]
08:09:02 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | After round 19/34: 
08:09:02 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Number of ranking changes: 74
08:09:02 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Sum of absolute ranking changes: 3878.0 (avg rank chg 22.95)
08:09:02 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | last_two: 22.585798816568047, prev_two: 26.988165680473372
08:09:02 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | ---------------------------------------------------
08:09:02 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Running round 20 of max 34
08:09:02 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | ---------------------------------------------------
08:09:02 | NewsletterAgent.test_newsletter_20251009073511683

,id,title,bradley_terry,bt_z
2,2,An overview of detailed AI usage reports from ...,11.119778,2.312772
55,55,AMD’s Landmark Deal with OpenAI Marks a New Ph...,10.429236,2.169148
80,80,Sam Altman says that ChatGPT has reached 800M ...,8.866090,1.844034
74,74,"As part of its deal with AMD, OpenAI will rece...",8.617536,1.792338
118,118,EU wants key sectors to use made-in-Europe AI,8.188451,1.703094
...,...,...,...,...
77,77,How an H-1B visa-holder went from dental schoo...,-8.147509,-1.694578
60,60,China Stocks Rise Post-Break as AI Buzz Outshi...,-8.349296,-1.736547
140,140,Ocean Protocol withdraws from AI token allianc...,-8.812920,-1.832975
41,41,‘I Believe It’s a Bubble’: What Some Smart Peo...,-9.215563,-1.916720


08:09:55 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Top 10 ids: [2, 55, 80, 74, 118, 31, 98, 64, 132, 120]
08:09:55 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | After round 20/34: 
08:09:55 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Number of ranking changes: 78
08:09:55 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Sum of absolute ranking changes: 4550.0 (avg rank chg 26.92)
08:09:55 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | last_two: 24.93491124260355, prev_two: 23.80473372781065
08:09:55 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Increase in avg rank change, stopping
08:09:55 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Low rated articles: 1
08:09:55 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | low rated article: 3 Top Artificial Intelligence (AI) Stocks Ready for a Bull Run -3.2449254283735973


⏱️  Total execution time: 1241.97s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 168 articles with average rating 6.0/10.
⭐ High quality articles (≥7.0): 63
💾 Ratings stored in persistent state.


In [30]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
2,Financial Times,"An overview of detailed AI usage reports from OpenAI and others, as Microsoft's AI for Good Lab estimates that 15% of the world's working population is using AI",13.356272,"Microsoft's AI for Good Lab estimates 15% of the world's workforce uses AI; ChatGPT handles over 18 billion weekly messages with >10% global adoption in three years, while major firms report large user bases (Meta 1 billion; Copilot 100 million)."
45,Yahoo,AMD’s Landmark Deal with OpenAI Marks a New Phase in the AI Chip Race,12.874937,"Reuters reported Oct. 6 that AMD has a multi-year deal to supply OpenAI with AI chips, potentially generating tens of billions annually, including a warrant enabling OpenAI to acquire up to 10% of AMD, with GPU deployment starting H2 2026."
1,The New York Times,What the Arrival of A.I. Video Generators Like Sora Means for Us,12.069659,"OpenAI released Sora, a free iPhone app that instantly generates realistic videos from text; while used creatively, it raises concerns that fabricated footage could spread disinformation and erode trust in video as proof."
24,Yahoo,TSMC posts forecast-beating Q3 revenue surge on AI boom,11.811078,"TSMC reported Q3 revenue rose 30% year‑on‑year to T$989.92 billion ($32.47 billion), beating LSEG SmartEstimate forecasts as surging AI‑related demand drove sales."
6,Financial Times,"OpenAI's deals with Nvidia, AMD, Oracle, and others for computing power have topped $1T, dwarfing its revenue and raising questions about how it can fund them",11.501945,"OpenAI has secured computing deals worth over $1 trillion with partners including Nvidia, AMD and Oracle, a scale that dwarfs its revenue and raises questions about how it will fund the agreements."
...,...,...,...,...
133,Deadline,"'Mad Max' Director George Miller Has ""Intense Curiosity"" About AI",1.527236,"Mad Max director George Miller says he welcomes AI's disruption to filmmaking, comparing its impact to oil paint and photography and arguing it will democratize production by enabling young filmmakers to create and compile footage without traditional financial barriers."
136,Biztoc,Kevin O'Leary Says AI Is The 'Best Investment' For Small Businesses — Calls Customer Service Bots A Secret Weapon,1.466460,"Investor Kevin O'Leary says artificial intelligence is the best investment for small businesses, calling AI-powered customer service bots a competitive weapon that can improve customer interactions, boost operational efficiency and accelerate adoption among small firms."
145,Biztoc,I Got Laid Off. Job Hunting in the Age of Robots Has Been a Pain,1.123615,"AI has transformed job hunting: tools like ChatGPT help craft resumes and cover letters, but widespread automated screening shifts recruitment dynamics and may disadvantage applicants unfamiliar with AI-driven methods, prompting experts to urge adaptation to remain competitive."
139,Reuters,Breakingviews - How Europe can play to its strengths with AI,0.943390,"Breakingviews argues Europe should leverage its regulatory strengths, industrial base and research capacity to shape AI development, protect privacy and remain competitive globally."


In [40]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


13.4 - [An overview of detailed AI usage reports from OpenAI and others, as Microsoft's AI for Good Lab estimates that 15% of the world's working population is using AI](https://t.co/JNcBxqSLyu) - Financial Times

Microsoft's AI for Good Lab estimates 15% of the world's workforce uses AI; ChatGPT handles over 18 billion weekly messages with >10% global adoption in three years, while major firms report large user bases (Meta 1 billion; Copilot 100 million).
    


12.9 - [AMD’s Landmark Deal with OpenAI Marks a New Phase in the AI Chip Race](https://finance.yahoo.com/news/amd-landmark-deal-openai-marks-101028972.html) - Yahoo

Reuters reported Oct. 6 that AMD has a multi-year deal to supply OpenAI with AI chips, potentially generating tens of billions annually, including a warrant enabling OpenAI to acquire up to 10% of AMD, with GPU deployment starting H2 2026.
    


12.1 - [What the Arrival of A.I. Video Generators Like Sora Means for Us](https://www.nytimes.com/2025/10/09/technology/personaltech/sora-ai-video-impact.html) - The New York Times

OpenAI released Sora, a free iPhone app that instantly generates realistic videos from text; while used creatively, it raises concerns that fabricated footage could spread disinformation and erode trust in video as proof.
    


11.8 - [TSMC posts forecast-beating Q3 revenue surge on AI boom](https://finance.yahoo.com/news/tsmc-posts-q3-revenue-32-053736693.html) - Yahoo

TSMC reported Q3 revenue rose 30% year‑on‑year to T$989.92 billion ($32.47 billion), beating LSEG SmartEstimate forecasts as surging AI‑related demand drove sales.
    


11.5 - [OpenAI's deals with Nvidia, AMD, Oracle, and others for computing power have topped $1T, dwarfing its revenue and raising questions about how it can fund them](https://t.co/ZF1E09CAyV) - Financial Times

OpenAI has secured computing deals worth over $1 trillion with partners including Nvidia, AMD and Oracle, a scale that dwarfs its revenue and raises questions about how it will fund the agreements.
    

...


1.5 - ['Mad Max' Director George Miller Has "Intense Curiosity" About AI](https://deadline.com/2025/10/mad-max-george-miller-welcomes-ai-1236574232/) - Deadline

Mad Max director George Miller says he welcomes AI's disruption to filmmaking, comparing its impact to oil paint and photography and arguing it will democratize production by enabling young filmmakers to create and compile footage without traditional financial barriers.
    


1.5 - [Kevin O'Leary Says AI Is The 'Best Investment' For Small Businesses — Calls Customer Service Bots A Secret Weapon](https://biztoc.com/x/d6adf3c8e76ae1c3) - Biztoc

Investor Kevin O'Leary says artificial intelligence is the best investment for small businesses, calling AI-powered customer service bots a competitive weapon that can improve customer interactions, boost operational efficiency and accelerate adoption among small firms.
    


1.1 - [I Got Laid Off. Job Hunting in the Age of Robots Has Been a Pain](https://biztoc.com/x/2973b311524c0f88) - Biztoc

AI has transformed job hunting: tools like ChatGPT help craft resumes and cover letters, but widespread automated screening shifts recruitment dynamics and may disadvantage applicants unfamiliar with AI-driven methods, prompting experts to urge adaptation to remain competitive.
    


0.9 - [Breakingviews - How Europe can play to its strengths with AI](https://www.reuters.com/commentary/breakingviews/how-europe-can-play-its-strengths-with-ai-2025-10-08/) - Reuters

Breakingviews argues Europe should leverage its regulatory strengths, industrial base and research capacity to shape AI development, protect privacy and remain competitive globally.
    


0.7 - [Prediction: While Big Tech Grabs Headlines, These Small AI Stocks Could Outperform](https://biztoc.com/x/601985544b73b680) - Biztoc

Analysts predict smaller AI-focused stocks could outperform megacap tech firms by leveraging niche innovations and market opportunities, despite big tech’s current dominance via large investments and profitable advances.
    

In [45]:
from utilities import send_gmail
# Create HTML content for email
html_items = []
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    rating = getattr(row, 'rating', 0)
    title = getattr(row, 'title', 'No Title')
    url = getattr(row, 'url', '#')
    site_name = getattr(row, 'site_name', 'Unknown')
    short_summary = getattr(row, 'short_summary', '')

    html_item = f"""
    <div style="margin-bottom: 20px; padding: 10px; border-left: 3px solid #4CAF50;">
        <h3 style="margin: 0 0 5px 0;">{rating:.1f} - <a href="{url}">{title}</a> - {site_name}</h3>
        <p style="margin: 5px 0 0 0; color: #666;">{short_summary}</p>
    </div>
    """
    html_items.append(html_item)

html_content = "\n".join(html_items)

# Create subject with date/time
now = datetime.now()
subject = f"AI news items - {now.strftime('%Y-%m-%d %H:%M:%S')}"

# Send email
send_gmail(subject, html_content)



In [28]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.tiame()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - starat_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:12:41 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
08:12:41 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Using model 'gpt-4.1-mini' for topic extraction
08:12:41 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Processing 168 articles for topic extraction
08:12:41 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Processing 17 chunks with concurrency 16


▶ Starting Step 6: step_06_cluster_by_topic
concurrency:  16


08:12:46 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Successfully extracted 654 total topics across articles
08:12:46 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 

=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       34
=== Clustering Quality Metrics ===
Number of clusters: 18
Noise points: 79 (47.0%)
Average cluster size: 4.9 ± 3.4
Cluster size range: 2 - 16
=== Quality Scores ===
Silhouette Score: 0.253 (higher is better)
Calinski-Harabasz Score: 7.8 (higher is better)
Davies-Bouldin Score: 1.223 (lower is better)
HDBSCAN Validity Index: 0.107
Composite Score: 0.180 (higher is better)

[I 2025-10-09 08:13:50,753] Trial 194 finished with value: -0.1800747069854287 and parameters: {'n_components': 34, 'min_cluster_size': 2, 'min_samples': 2}. Best is trial 76 with value: -0.22368133148079047.
=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       76
=== Clustering Quality Metrics ===
Number of clusters: 14
Noise points: 107 (63.7%)
Average cluster size: 4.4 ± 2.3
Cluster size range: 2 - 10
=== Quality Scores ===
Silhouette Score: 0.230 (higher is better)
Calinski-Harabasz S

INFO:llm:Successfully retrieved prompt 'newsagent/topic_writer' from Langfuse
INFO:llm:Parsed prompt 'newsagent/topic_writer': model=gpt-4.1, system_len=377, user_len=57
08:13:52 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | 1: Artificial Intelligence Business and Societal Impact
08:13:52 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Six companies pushing the legal world into the AI era (Legal Technology, AI Automation, Legal Research, Contract Management, Mergers and Acquisitions, Enterprise Software, Deals)
What the Arrival of A.I. Video Generators Like Sora Means for Us (Video Generators, Gen AI, Disinformation, Media Verification, Deepfakes, OpenAI, Privacy And Surveillance)
An overview of detailed AI usage reports from OpenAI and others, as Microsoft's AI for Good Lab estimates that 15% of the world's working population is using AI (Gen AI, AI Adoption, Chatbots, Coding Assistants, Virtual Assistants, Business AI, Government AI)
Sources: Microso

08:13:52 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | 

08:13:53 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | 0: SoftBank Acquires ABB Robotics Division
08:13:53 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | SoftBank Group agrees to acquire ABB's robotics business, which generated $2.3B in sales in 2024, for $5.4B; ABB expects the deal to close in mid-to-late 2026 (SoftBank Group, ABB Robotics, Robotics Acquisition, Physical AI, Mergers and Acquisitions, Hardware, Automation)
Billionaire Masayoshi Son’s SoftBank Deepens AI Push With $5.4 Billion ABB Robotics Deal (SoftBank Robotics Acquisition, ABB Robotics Unit, AI and Robotics Strategy, Industrial Automation, Mergers and Acquisitions, AI Hardware, Masayoshi Son)
SoftBank to buy ABB’s robotics arm in $5.4 billion deal (SoftBank, ABB, Industrial Robots, Robotics Acquisition, Automation, Mergers and Acquisitions, Manufacturing)
08:13:53 | NewsletterAgent.test_newsletter_20251009073

⏱️  Total execution time: 72.43s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 168 articles into topic clusters.


In [29]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:14:35 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179


▶ Starting Step 7: step_07_select_sections


08:16:16 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Cleaning up initial categories: ['AMD OpenAI Chip Deal', 'SoftBank Acquires ABB Robotics', 'Artificial Intelligence Business and Societal Impact', 'Gaming Mouse Speech Exploit', 'AI Lawsuit Settlement Funding', 'SoftBank Acquires ABB Robotics Division', 'Deloitte Refund Over AI Errors', 'Other', 'OpenAI AgentKit Launch', 'EU AI Sovereignty Strategy', 'AI Value Realization Gap', 'ChatGPT 800M Weekly Users', 'EU AI Rules Slow Startups', 'AI Music Industry Disruption', 'TSMC Revenue Boom From AI', 'OpenAI Platform Monetization', 'CoreWeave Serverless RL', 'Threat Groups Use AI Tools', 'US AI Policy Patchwork', 'ChatGPT Go Asia Expansion', 'ChatGPT Apps Platform Play', 'Agent Memory Frameworks', 'AstraZeneca $555M AI Deal', 'AI Agent Security Mitigation', 'AI In Legal Services', 'Microsoft Copilot Healthcare Push', 'Instant AI Video Generators', 'Global AI Adoption Surge', 'AI Startup Funding Surge', 'CodeMender Auton

08:18:00 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Handling singleton categories
08:18:00 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Singleton categories (4): ['n8n $2.5B Valuation', 'US Approves Nvidia UAE Exports', 'SoftBank Acquires ABB Robotics', 'AMD OpenAI Chip Deal']
08:18:00 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Non-singleton categories (17): ['AI Societal Impact', 'AI Adoption And Funding Surge', 'Other', 'AI Business Value Gap', 'AI Legal Services', 'Instant AI Video Generators', 'OpenAI AgentKit Launch', 'TSMC AI Revenue Boom', 'Agent Memory Frameworks', 'EU AI Sovereignty Strategy', 'OpenAI Massive Compute Deals', 'AI Music Industry Disruption', 'ChatGPT Asia Expansion', 'US AI Policy Patchwork', 'ChatGPT Apps Platform Play', 'AI Lawsuit Settlement Funding', 'Threat Actors Use AI Tools']
08:18:00 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Reassigning 4 singleton articles to non-singlet

⏱️  Total execution time: 212.45s
📊 Final result:
Categories and article counts:
cat
AI Adoption And Funding Surge    34
AI Business Value Gap            12
AI Lawsuit Settlement Funding     2
AI Legal Services                 4
AI Music Industry Disruption      2
AI Societal Impact               42
Agent Memory Frameworks           3
ChatGPT Apps Platform Play        2
ChatGPT Asia Expansion            2
EU AI Sovereignty Strategy        3
Instant AI Video Generators       4
OpenAI AgentKit Launch            4
OpenAI Massive Compute Deals      4
Other                            20
TSMC AI Revenue Boom              3
Threat Actors Use AI Tools        2
US AI Policy Patchwork            3


In [10]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


11:16:17 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Drafting sections for 17 categories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1995, user_len=22
11:16:17 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Initialized LLMagent:
system_prompt: You are a newsletter editor transforming a collection of raw news stories into a compelling, coherent topic summary.

# TASK
Transform the list of news stories into a well-structured newsletter section with a strong title, crisp headlines, and punchy summaries.

# INPUT
- a list of json objects sorted by rating (highest first)
- Each item: { "rating": number, "short_summary": string, "site_name": string, "url": string }

# OUTPUT: minified JSON only (no code fences, no line breaks, no extra whitespace):
json{"section_title":"<string>",
     "stories":[{"headline":"<st

▶ Starting Step 8: step_08_draft_sections


11:16:17 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | User message: STORIES:
[{"rating":12.0696593332,"short_summary":"OpenAI released Sora, a free iPhone app that instantly generates realistic videos from text; while used creatively, it raises concerns that fabricated footage could spread disinformation and erode trust in video as proof.","site_name":"The New York Times","url":"https:\/\/www.nytimes.com\/2025\/10\/09\/technology\/personaltech\/sora-ai-video-impact.html"},{"rating":8.2701316696,"short_summary":"Creative Artists Agency warns Microsoft-backed OpenAI\u2019s new AI video generator Sora risks artists\u2019 works being used without credit or compensation; OpenAI CEO Sam Altman says controls will let rights holders manage usage and share revenue.","site_name":"Hacker Noon","url":"https:\/\/hackernoon.com\/caa-warns-openais-sora-puts-artists-at-risk-demands-credit-and-compensation-controls?source=rss"},{"rating":4.9938176676,"short_summary":"Goldfinch and Th

11:16:17 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | User message: STORIES:
[{"rating":10.8763449081,"short_summary":"OpenAI and Anthropic are considering using investor funds to settle potential multibillion-dollar lawsuits after insurers, unable to assess the scale of emerging AI-related liabilities, declined to underwrite coverage, prompting calls for new risk-assessment models and insurance products.","site_name":"Financial Times","url":"https:\/\/t.co\/j24xWrZqlJ"},{"rating":8.8512622271,"short_summary":"FT reports OpenAI and Anthropic are considering using investor funds and self\u2011insurance (including a captive) to cover potential multibillion\u2011dollar copyright suits; OpenAI has about $300 million of AI risk insurance, while Anthropic faces a preliminarily approved $1.5 billion settlement.","site_name":"Yahoo","url":"https:\/\/finance.yahoo.com\/news\/openai-anthropic-eye-investor-funds-040818912.html"}]
11:16:17 | NewsletterAgent.test_newsletter_20251

11:16:17 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | User message: STORIES:
[{"rating":12.8749372208,"short_summary":"Reuters reported Oct. 6 that AMD has a multi-year deal to supply OpenAI with AI chips, potentially generating tens of billions annually, including a warrant enabling OpenAI to acquire up to 10% of AMD, with GPU deployment starting H2 2026.","site_name":"Yahoo","url":"https:\/\/finance.yahoo.com\/news\/amd-landmark-deal-openai-marks-101028972.html"},{"rating":11.5019448241,"short_summary":"OpenAI has secured computing deals worth over $1 trillion with partners including Nvidia, AMD and Oracle, a scale that dwarfs its revenue and raises questions about how it will fund the agreements.","site_name":"Financial Times","url":"https:\/\/t.co\/ZF1E09CAyV"},{"rating":7.295615744,"short_summary":"OpenAI is positioning itself as the Windows of AI, securing partnerships with Oracle, Nvidia, Samsung, SK hynix and a multibillion-dollar AMD deal, embedding apps int

...

11:16:41 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Result: section_title='Insurers Balk at AI Liability' stories=[SectionStory(headline='OpenAI, Anthropic eye self-insurance for multibillion-dollar suits', summary='Insurers balk at AI liabilities, prompting OpenAI and Anthropic to self-insure with investor funds; OpenAI holds ~$300m coverage; Anthropic faces preliminarily approved $1.5b settlement.', links=[SectionStoryLink(url='https://t.co/j24xWrZqlJ', site_name='Financial Times'), SectionStoryLink(url='https://finance.yahoo.com/news/openai-anthropic-eye-investor-funds-040818912.html', site_name='Yahoo')], prune=False)]
11:16:43 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Result: section_title='Making AI Remember and Reason Better' stories=[SectionStory(headline='ReasoningBank gives AI agents reusable memory', summary='Illinois and Google unveil ReasoningBank, a memory for reusable reasoning from wins and failures, boosting complex task perfo

11:16:49 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Result: section_title='Lights, Camera, AI' stories=[SectionStory(headline='OpenAI launches Sora iPhone app for instant AI video', summary='OpenAI releases free Sora iPhone app generating realistic videos from text, sparking creativity, and fears of deepfake disinformation eroding trust in video evidence.', links=[SectionStoryLink(url='https://www.nytimes.com/2025/10/09/technology/personaltech/sora-ai-video-impact.html', site_name='The New York Times')], prune=False), SectionStory(headline='CAA warns Sora exploits artists; OpenAI promises controls', summary="Creative Artists Agency says Sora risks uncredited, unpaid use of artists' work. CEO Sam Altman pledges rights controls and revenue sharing for creators.", links=[SectionStoryLink(url='https://hackernoon.com/caa-warns-openais-sora-puts-artists-at-risk-demands-credit-and-compensation-controls?source=rss', site_name='Hacker Noon')], prune=False), SectionStory(hea

11:19:38 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Result: section_title='AI Power Plays' stories=[SectionStory(headline='China pulls ahead in AI robotics', summary='Schmidt-backed analysis says China leads in AI-driven robotics, outpacing the U.S., with implications for economic power, supply chains, and strategic balance.', links=[SectionStoryLink(url='https://www.washingtonpost.com/opinions/2025/10/09/china-robotics-humanoid-technology-advances/', site_name='The Washington Post')], prune=False), SectionStory(headline='Self-replicating AI could supercharge growth', summary='RAND model: autonomous, self-replicating AI adds 3.8 percentage points to annual GDP growth, making the economy nearly 4x by 2045 if alignment and safety challenges are solved.', links=[SectionStoryLink(url='https://www.rand.org/pubs/research_reports/RRA4220-1.html', site_name='RAND Corporation')], prune=False), SectionStory(headline='U.S. intel pivots to AI open-source', summary="Report: U.S

11:19:42 | NewsletterAgent.test_newsletter_20251009073511683688 | INFO | Result: section_title='AI Money Moves' stories=[SectionStory(headline='AI Hits 15% of Global Workforce', summary='Microsoft lab says 15% of workers use AI. ChatGPT handles 18B weekly messages; Meta 1B users; Copilot 100M, three years post-launch.', links=[SectionStoryLink(url='https://t.co/JNcBxqSLyu', site_name='Financial Times')], prune=False), SectionStory(headline='AI Startups Raise $3.2B in a Week', summary='Sept 28–Oct 4 saw $3.2B raised, led by Cerebras’ $1.1B Series G, Vercel’s $300M Series F, and a $300M autonomous AI labs seed.', links=[SectionStoryLink(url='https://hackernoon.com/weekly-ai-startup-funding-september-28-to-october-4-2025?source=rss', site_name='Hacker Noon')], prune=False), SectionStory(headline='SoftBank Buys ABB Robotics for $5.4B', summary='ABB’s robotics unit with $2.3B sales will be sold to SoftBank; closing expected mid-to-late 2026 as ABB refocuses on electrification and automation

✅ Completed Step 8: Drafted 17 sections with 3345 total words
⏱️  Total execution time: 205.19s
📊 Final result:
Drafted 17 sections with 3345 total words


In [ ]:
give section a standard string output
output sections
do the pruning 
rewrite other after pruning


In [66]:
input_text = headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")
input_text
ascending
a

'[{"rating":9.5199365,"short_summary":"McKinsey finds only 30% of AI software vendors report quantifiable ROI from deployments; rising IT costs without matching labor cuts, weak change-management investment and complex pricing hinder adoption, prompting vendors to adopt hybrid per-user plus consumption-based pricing.","site_name":"The Register","url":"https:\\/\\/www.theregister.com\\/2025\\/10\\/09\\/mckinsey_ai_monetization\\/"},{"rating":9.4274351938,"short_summary":"The Bank of England warns global financial markets could face a sharp correction if investor sentiment sours on AI prospects or confidence in the U.S. Federal Reserve\'s independence, citing inflated share prices driven by AI enthusiasm.","site_name":"Biztoc","url":"https:\\/\\/biztoc.com\\/x\\/42d9125aea870810"},{"rating":9.0760579343,"short_summary":"BCG finds only 5% of over 1,250 global companies realize significant AI returns; software, telecommunications, and fintech lead, driven by long-term AI planning, leadersh

In [68]:
write_section_system_prompt, write_section_user_prompt, model = \
    LangfuseClient().get_prompt("newsagent/write_section")

write_section_agent = LLMagent(
    system_prompt=write_section_system_prompt,
    user_prompt=write_section_user_prompt,
    output_type=Section,
    model=model,
    verbose=True,
    logger=logger
)

response = await write_section_agent.run_prompt(input_text=input_text)
response



INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1995, user_len=22
11:04:47 | NewsletterAgent.newsletter_agent | INFO | Initialized LLMagent:
system_prompt: You are a newsletter editor transforming a collection of raw news stories into a compelling, coherent topic summary.

# TASK
Transform the list of news stories into a well-structured newsletter section with a strong title, crisp headlines, and punchy summaries.

# INPUT
- a list of json objects sorted by rating (highest first)
- Each item: { "rating": number, "short_summary": string, "site_name": string, "url": string }

# OUTPUT: minified JSON only (no code fences, no line breaks, no extra whitespace):
json{"section_title":"<string>",
     "stories":[{"headline":"<string>",
                 "summary":"<string>",
                 "links":[{"url":"<string>",
                             "site_na

11:05:44 | NewsletterAgent.newsletter_agent | INFO | Result: section_title='AI ROI: Hype Meets Reality' stories=[Story(headline='McKinsey: Only 30% of AI Vendors Show ROI', summary='Report finds rising IT costs, weak change management, and complex pricing. Vendors pivot to hybrid per-user plus consumption models.', links=[Link(url='https://www.theregister.com/2025/10/09/mckinsey_ai_monetization/', site_name='The Register')], prune=False), Story(headline='BCG: Just 5% of Firms See Significant AI Returns', summary='Software, telecom, and fintech lead. Success correlates with long-term plans, leadership engagement, business–IT co-ownership, workforce upskilling, and strong data governance.', links=[Link(url='https://www.businessinsider.com/industries-seeing-value-from-ai-bcg-consulting-report-2025-10', site_name='Business Insider')], prune=False), Story(headline='CFOs Turn AI Into Measurable Finance ROI', summary='AI improves cash flows, forecasting, reconciliation, SQL queries, and month

Section(section_title='AI ROI: Hype Meets Reality', stories=[Story(headline='McKinsey: Only 30% of AI Vendors Show ROI', summary='Report finds rising IT costs, weak change management, and complex pricing. Vendors pivot to hybrid per-user plus consumption models.', links=[Link(url='https://www.theregister.com/2025/10/09/mckinsey_ai_monetization/', site_name='The Register')], prune=False), Story(headline='BCG: Just 5% of Firms See Significant AI Returns', summary='Software, telecom, and fintech lead. Success correlates with long-term plans, leadership engagement, business–IT co-ownership, workforce upskilling, and strong data governance.', links=[Link(url='https://www.businessinsider.com/industries-seeing-value-from-ai-bcg-consulting-report-2025-10', site_name='Business Insider')], prune=False), Story(headline='CFOs Turn AI Into Measurable Finance ROI', summary='AI improves cash flows, forecasting, reconciliation, SQL queries, and month-end close speeds, shifting finance from cost center

In [55]:
class Link(BaseModel):
    url: str = Field(description="URL of the article")
    site_name: str = Field(description="Name of the website/source")


class Story(BaseModel):
    headline: str = Field(description="Headline for the story")
    summary: str = Field(description="Summary of the story")
    links: List[Link] = Field(description="List of links related to this story")
    prune: bool = Field(description="Whether to prune/exclude this story")


class Section(BaseModel):
    section_title: str = Field(description="Title of the newsletter section")
    stories: List[Story] = Field(description="List of stories in this section")
        

headline_df=state.headline_df
headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")


In [41]:
print("# SUGGESTED TOPICS:")
catcount = headline_df.groupby("cat").count().reset_index()[['cat', 'source']].sort_values('source', ascending=False)
for c in catcount["cat"]:
    print(c)
print()
print("# RAW NEWS ITEMS:")
i =0
for row in headline_df.sort_values(["cat", "rating"], ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"Rating: {row.rating:.1f}\n")    
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")
    i +=1


# SUGGESTED TOPICS:
AI Societal Impact
AI Adoption And Funding Surge
Other
AI Business Value Gap
Instant AI Video Generators
AI Legal Services
OpenAI Massive Compute Deals
OpenAI AgentKit Launch
TSMC AI Revenue Boom
US AI Policy Patchwork
EU AI Sovereignty Strategy
Agent Memory Frameworks
ChatGPT Apps Platform Play
AI Music Industry Disruption
AI Lawsuit Settlement Funding
Threat Actors Use AI Tools
ChatGPT Asia Expansion

# RAW NEWS ITEMS:
[U.S. Approves Nvidia AI Chip Exports to UAE Under Bilateral Tech Pact](https://hackernoon.com/us-approves-nvidia-ai-chip-exports-to-uae-under-bilateral-tech-pact?source=rss) - Hacker Noon

Topics: Nvidia, AI Chip Exports, US Middle East Relations, Policy And Regulation, Governance, Deals, Data Centers

Rating: 10.1

The U.S. authorized Nvidia to export advanced AI chips worth billions to the UAE under a bilateral AI agreement signed earlier this year, enabling reciprocal UAE investments in the U.S. and support for large-scale data centers to advanc

In [42]:
headline_df['cat'].unique() 


array(['AI Legal Services', 'Instant AI Video Generators',
       'AI Adoption And Funding Surge', 'AI Lawsuit Settlement Funding',
       'AI Business Value Gap', 'Other', 'OpenAI Massive Compute Deals',
       'EU AI Sovereignty Strategy', 'AI Music Industry Disruption',
       'Agent Memory Frameworks', 'ChatGPT Asia Expansion',
       'US AI Policy Patchwork', 'ChatGPT Apps Platform Play',
       'AI Societal Impact', 'TSMC AI Revenue Boom',
       'OpenAI AgentKit Launch', 'Threat Actors Use AI Tools'],
      dtype=object)

In [ ]:
astate.get_completed_steps() 



In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [29]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"{row.rating:.1f}"))   
    display(Markdown(f"[{row.title}]({row.url}) - {row.site_name}".replace("$","\\\\$")))
    row_topics = ", ".join(row.topics)
    display(Markdown(f"Topics: {row_topics}".replace("$","\\\\$")))
    display(Markdown(f"{row.short_summary}".replace("$","\\\\$")))
    display(Markdown(f"{row.summary}".replace("$","\\\\$")))
    i +=1
#     if i>=30:
#         break

16.3

[Bank of England warns AI stock bubble rivals 2000 dotcom peak](https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/) - Ars Technica

Topics: Tech Stocks, Market Concentration, Finance, Policy and Regulation, Bubble, UK, Gen AI

Bank of England warns AI-driven valuations resemble the 2000 dotcom peak, noting Nvidia, Microsoft, Apple, Amazon and Meta account for 30% of the S&P 500—the highest market concentration in 50 years—and could spill over into Britain’s financial system.

- The Bank of England warns that the current AI-driven stock market valuations resemble the peak of the 2000 dotcom bubble, signaling increased risk of a sharp market correction.
- AI-focused companies constitute an unprecedented portion of market value, with five major firms—Nvidia, Microsoft, Apple, Amazon, and Meta—accounting for 30% of the S&P 500's valuation, marking the highest concentration in 50 years.
- The Bank of England highlighted that a negative shift in investor sentiment toward AI could have material spillover effects on Britain’s financial system, emphasizing concerns about market stability amid soaring AI investments.

...

0.6

[When Curl Works but IntelliJ Doesn't: The Ollama Connection Mystery](https://blog.tymscar.com/posts/intellijollamaconnectionmystery/) - 

Topics: IntelliJ, JVM Network, Programming Debugging, Networking, Java Network, Gen AI, Coding Assistants

IntelliJ failed to connect to a locally reachable Ollama instance because the JVM attempted IPv6 on an IPv4-only LAN, producing No route to host; adding -Djava.net.preferIPv4Stack=true forced IPv4 and restored connectivity without other changes.

- IntelliJ IDEA failed to connect to the Ollama instance despite curl successfully accessing it over both HTTPS and HTTP, due to JVM network stack behavior.
- The root cause was the JVM attempting an IPv6 connection first on an IPv4-only LAN, resulting in a "No route to host" error; this was resolved by adding the JVM option -Djava.net.preferIPv4Stack=true.
- The fix disabled IPv6 usage in the JVM, allowing IntelliJ to connect instantly to Ollama without changes to Traefik, TLS certificates, or network configuration, highlighting the importance of checking JVM network preferences in similar Java application connectivity issues.

In [ ]:
class DistilledStory(BaseModel):
    """DistilledStory class for structured output distillation into a single sentence """
    item: str = Field(description="List of StoryRating")
        
system, user, model = LangfuseClient().get_prompt("newsagent/item_distiller")

distill_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=DistilledStory,
            model=model,
            verbose=False,
            logger=logger
        )

In [ ]:
response = await distill_agent.run_prompt(input_text="""AI 'Homeless Man' Challenge Sparks Outrage as Police Called Over Dangerous Viral Trend - International Business Times

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Rating: 1.5

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Parents in shock after dangerous 'homeless man' Snapchat AI prank goes horribly wrong, police called to calm the viral chaos.

A viral AI-driven prank called the 'homeless man' challenge on Snapchat deceived parents into thinking a homeless person had broken into their home, prompting police intervention.
The prank caused significant public backlash due to the misuse of AI technology to create real emergency scares, leading to police questioning the pranksters and debates about the legal consequences.
Experts emphasize the ethical concerns and potential emotional harm caused by such digital pranks, highlighting the need for responsible use of AI to avoid wasting emergency resources and creating community panic.
""")
response

In [19]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")

    i +=1

[SoftBank to buy ABB’s robotics unit in \\$5.4bn deal](https://www.ft.com/content/4080ecae-0654-4597-9596-025c373874c3) - Financial Times

Topics: SoftBank, Robotics, Mergers and Acquisitions, Artificial Superintelligence, Hardware, Manufacturing, Japan

SoftBank will acquire ABB's robotics unit for \\$5.4 billion, aligning with founder Masayoshi Son's vision to advance artificial superintelligence and expand SoftBank's robotics and AI capabilities.

- SoftBank is set to acquire ABB's robotics unit for \\$5.4 billion as part of its strategic expansion into robotics technology.
- The acquisition supports SoftBank founder Masayoshi Son's vision of achieving 'artificial superintelligence,' reflecting significant business growth and innovation ambitions.
- This deal positions SoftBank as a major player in the robotics market, with expected implications for advancing AI-integrated robotic solutions across industries.

~~~
[Google’s latest AI model uses a web browser like you do](https://www

TODO:
- update final prompt
- output sections using short summary
# SUGGESTED TOPICS
AI Agents And Reliability
AI Creative Industry Impact
AI Development Tools And Standards
AI Market Valuations
AI Phishing Surge
AI Security Risks
AI Workforce Impact
C2PA Image Provenance
Circular Deal Inflation
Cross-Industry AI Adoption
Crunch Lab Decentralized AI
Data Center Environmental Impact
Deepfake Video Ethics
Dell Raises AI Forecasts
EU AI Strategy
Enterprise AI Data Leakage
Enterprise AI Partnerships
Google Gemini 2.5
Healthcare AI Investments
OpenAI Platform Issues
Other
Qualcomm Acquires Arduino
SoftBank Acquires ABB Robotics
Youth Support AI Ethics

- take each summary and boil it down to 1 sentence , output correct format
- initial write sections - prompt and output json for each section asynchronously
- check and rewrite each section for format asynchronsously
- assemble sections
- do a critic loop